# Project Journal

This notebook serves to be a journal of the project. It shall present interesting and important results of the project.

### Dataset choice

The first task in this project is to choose the dataset and extract the appropriate data. There are several datasets that can be used for this project. The three main ones are presented below:

#### 1. Million Song Dataset
This dataset was presented by Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. *The Million Song Dataset. In Proceedings of the 12th International Society for Music Information Retrieval Conference (ISMIR 2011), 2011*.

This dataset has about 1,000,000 tracks from 44,745 different artists. The audio here is **not** directly accessibly. However, it can be obtained through 7digital API. Through contacting Lee et al. from *Sample-Level Deep Convolutional Neural Networks for Music Auto-Tagging using Raw Waveforms*, FTP access to all the files was achieved. However, this dataset ranges in the size of about **100TB** which is relatively unfeasible to download and use on a 3 month MSC project. This option is set aside to be used as backup if none of the other datasets are available to use.

#### 2. Magnatagatune Dataset
This dataset was presented by Edith Law, Kris West, Michael Mandel, Mert Bay and J. Stephen Downie. *Evaluation of algorithms using games: the case of music annotation. In  Proceedings of the 10th International Conference on Music Information Retrieval (ISMIR 2009), 2009*. 

This dataset has about 25,863 tracks from 230 different artists. In this dataset the audio is available. It can be seen that this dataset does not have a large variety of artists whilst being relatively small. However, this is the simplest to use as it has both the music and tags ready available. This is considered to be the primary option as a dataset.

#### 3. FMA dataset
This is a new dataset presented in April 2017 by Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst and Xavier Bresson. *FMA: A Dataset for Music Analysis. In arXiv 2017.

This dataset has about 106,574 tracks from 16,341 different artists. It has audio available. However, in this case specific tags apart from the genre, era and artist name are not available. This are the tags that were actually needed for the task. This was seen as a better dataset from the Magnatagatune Dataset. Two methods were attempted to obtain the appropriate tags. 

##### 3.1 Method one Obtain tags through  Last.FM sqlite tag database
The tags from the Last.FM sqlite tag database were attempted to be extracted. These tags are partly used the MSD dataset. These were obtained both by hand and using the Last.FM API interface. However, in this database, there is no reference between the track id and the track name. This prove to be a problem since the only way of finding such a relationship was to sift through all the seperate JSON files and find the matching name to track id, and then extract the tags. This would be very computationally intensive since, there are ~100,000 songs in the FMA and the Last.FM database has ~500,000 tracks. Each specific track needs to be found from a folder with ~500,000 JSON files.

##### 3.2 Use Last.FM API
The second option was to use the Last.FM API. A function called lastfm_apireq.py was setup with a personal Last.FM account that can use API calls to ask for the tracks and extract the tags. However, from a sample of 100, more than 75% of the tracks had no tags. 
On thing that could be done instead is to check for album tags instead. However, these would not be track specific. 

Instead it was proposed that the FMA dataset is used prior to doing the tests using the MTAT dataset in order to find the difference in perfomance between the datasets. This could be done as shown by Lee et al. in *Sample-Level Deep Convolutional Neural Networks for Music Auto-Tagging using Raw Waveforms* when they compared the MTAT with MSD.

### Dataset Extraction

In order to extract the data from the magnatatatune dataset, a function extract_ds.py was created. This funciton does the following:

1. Extract metadata - Extract tags, mp3 filenames, track ids, and label map.
2. Shuffle data - Shuffle the data according to the seed provided in the package __init__.py file.
3. Reduction of samples - Reduce the number of files (if needed). If no reduction -1 is provided.
4. Sort tags - The targets and label map is sorted according to the frequency of tags.
5. Extract data - The data is extracted from the mp3 files and saved as numpy arrays in tracks folder.
6. Seperate and merge - Metadata is separated into 3 archives. Train, valid, test in fractions.
7. Save metadata archives - Metadata is saved as archives that can be loaded and tids used to load the numpy music arrays.

The extract_ds.py is provided in the pydst package. Logger logs info level information to ext_ds_XX_XX.log files in the log folder.

Total Runtime: **52:24**

Files that given error: **3** 

Reason: **The file is corrupted as it is 0 Bytes**

Total size of tracks folder: **~45GB**

### Data Provider

One of the problems of the data provider is that the dataset is considerably large. If the raw sample data is kept, this is about 45GB which is not possible to load in memory. If down sampling is performed 10:1 samples, the dataset would reduce to 9.5GB which is still very large and not ideal to load in memory. 

One of the methods to solve this is to load batchwise. However, if this is done naively, there would be an additional computational cost which includes calling for the data before running the tensorflow session on the graph. Hence, the runs over all the epochs would require about (train_size/batch_size)\*epochs number of disk calls. This is a considerably overhead.

The solution opted for this scenario is to use a FIFO Queue with a seperate thread to load the queue and another to run the tensorflow graph. Hence, using this method, the graph trianing wouldn't slow down due to the loading of the input data. The data would be available in the queue which the enqueing thread takes care of in the background.

Since now some modules such as the tensorflow FIFOQueue are used, the data provider can also be part of the graph model. Thus one session for all the graph can be used.


### Graph Construction

At first, i was going to construct a class that enables a graph to be defined and layers are added with simple instructions such as 'add_ffl' to add specific layers. However, one of the problems encountered here was that in order for the graph to be trained it needs variables such as error, accuracy and train_step which are graph native. 

Even though construction of such class is still possible, it was seen as an added unneeded abstraction. The tensorflow tf.layers module provides similar methods such as 'tf.layers.dense' that are very simple to use. Even though a simple graph can be naively constructed and trained, a few approaches were considered in order for the implementation to be more intuative and training time evaluation to be performed.


#### Design considerations:

#### 1. Tensorflow graph is constructed in a function.
The tensorflow graph was opted to be constructed in a function for various reasons. The first reason is for many graphs of the same structure to be defined easily. Hence if *bgraph_t1* is a four mlp model, this can be defined as many times as needed. 

Another important aspect is that the graph can be defined that the variables are shared. In any data model, it is a good idea that evaluation is performed periodically while training in order to determine if the model is overfitting or not. However, since the data this time is supplied by a queue whcih loads the data via a seperate thread this is not as simple as introducing another for loop as when the entire dataset is loaded in memory. 

###### a. After a number of epochs turning of one dataprovider and introducing another.
This approach works intuitively, but has a big drawback. Even though this could be done using a flow manipulating tensorflow function, this methods needs the graph to be modified mid-experiment which is not ideal. Furthermore, this would add extra processing on the training graph which continues to increase the computation time. 

###### b. Have two seperate models one for training and one for evaluation.
This essentially works and is the fastest option. A model is trained, periodically checkpoints the variables into a binary file, which are then loaded by the evaluation process and the model is evaluated. However, here the drawback is that double the memory of the original graph is needed. Even though on CPU this might not be a big problem, on GPUs where memory is limited this could create overflow exceptions. Hence, this method would greatly limit the sizes of the networks that can be possibly trained. 

###### c. Have two models with shared variables.
This is the approach considered to be the best. Tensorflow offers methods that enables sharing of variables. Hence, two graphs using the same variables can run, one for training and another for evaluations. However, in this case, native tensorflow functions need to be used in order to avoid, race conidtions and loading of out of data variables. When the function sess.run(...) is called, a snapshot of the variables is saved and used for processing. Thus there is no variables updating dynamically which makes the system safe to use on seperate threads or processes.



## Experiments

### 1. Dieleman et al.'s convolutional neural network
##### Experiment 1
In the first experiment involved:
1. First strided convolutional Layer 256 filter length, 256 stride, 1 feature depth
2. Second and Third convolutional Layer 8 Filter Length, 8 stride, 32 feature depth
3. Between first and second layer, maxpool layer with length and stride of 4
4. Fully connected final two layers with 100 and 50 neurons respectively

Training time: 30mins<br>
_Notes_: <br>
The whole raw signal was used to train the network. This is different from what is presented by dieleman et al. Dieleman et al. seperated the raw audio into small 3 second windows and then performed the training. What was analysed in this setup is that the nerwork did not have enough flexibility to learn the raw music features. The majority of the tags were pushed down to a probability of zero in order to reflect the fact that the targets are sparse. In fact, reflecting on this make is clear that trying to have a neural netowrk to compress 465984 changing integer values of information to give out 50 tags is very difficult. Especially since the first layer has only one feature depth and hence having limited capacity for information.
In factm the ROC remained at 0.5 and there were very few true positives for each tag indicating that allocation was rather random.

##### Experiment 2


### 1. Self designed basic convolutional nueral network.
##### Experiment 1
In the first experiment involved:
1. First convolutional Layer 16 filter length, 16 stride, 4 feature depth
2. Second convolutional Layer 8 Filter Length, 4 stride, 8 feature depth
3. Maxpool layer of pool length 2 and stride 2
4. Third convolutional Layer 4 Filter Length, 1 stride, 12 feature depth
5. Maxpool layer of pool length 2 and stride 2
6. Fully connected layer with 1000 hidden units
7. Fully connected layer with 300 hidden units
8. Output layer with 50 units

Training time: 80mins<br>
Test name: 
_Notes_: <br>
Unlike what was presented by Dieleman et al. this network is rather different. The first layers, filter length and and stride are much shorter and the depth was increased from 1 to 4. Furthermore, in the subsequent layers, the stride is reducred and the depth is increased to a maximum of 12 rather than 32. The aim of this network is instead of compressing all the information quickly, is to compress features in a gradual manner. Hence, the depth increased gradually and stride length decreased gradually. After test time, results were still not idea but some characteristics were noted. This time the total ROC increased to 0.54 instead of 0.5. Furthermore, some of the tags had rather good matching whilst others weren't. It could be analysed from mthe results that the tags such as **rock** (492/495) and **loud** (102/118) achieved good true positive values. On the other hand, tags such as **soft**, **ambient** and **classical** achieved poor results with the true positives down to zero. This highlights the fact that the latter tags, have audio values very low and there are only mellow changes in the signal which in turn can be difficult for the network to capture.